# Deep Q-Learning

In this notebook, you will implement a deep Q-Learning reinforcement algorithm. The implementation borrows ideas from both the original DeepMind Nature paper and the more recent asynchronous version:<br/>
[1] "Human-Level Control through Deep Reinforcement Learning" by Mnih et al. 2015<br/>
[2] "Asynchronous Methods for Deep Reinforcement Learning" by Mnih et al. 2016.<br/>

In particular:
* We use separate target and Q-functions estimators with periodic updates to the target estimator. 
* We use several concurrent "threads" rather than experience replay to generate less biased gradient updates. 
* Threads are actually synchronized so we start each one at a random number of moves.
* We use an epsilon-greedy policy that blends random moves with policy moves.
* We taper the random action parameter (epsilon) and the learning rate to zero during training.

This gives a simple and reasonably fast general-purpose RL algorithm. We use it here for the Cartpole environment from OpenAI Gym, but it can easily be adapted to others. For this notebook, you will implement 4 steps:

1. The backward step for the Q-estimator
2. The $\epsilon$-greedy policy
3. "asynchronous" initialization 
4. The Q-learning algorithm

To get started, we import some prerequisites.

In [1]:
%matplotlib inline

import gym
import numpy as np
import sys
import matplotlib.pyplot as plt
import time
import pickle

The block below lists some parameters you can tune. They should be self-explanatory. They are currently set to train CartPole-V0 to a "solved" score (> 195) most of the time. 

In [20]:
nsteps = 10001                       # Number of steps to run (game actions per environment)
npar = 16                            # Number of parallel environments
target_window = 200                  # Interval to update target estimator from q-estimator
discount_factor = 0.99               # Reward discount factor
printsteps = 1000                    # Number of steps between printouts
render = False                       # Whether to render an environment while training

epsilon_start = 1.0                  # Parameters for epsilon-greedy policy: initial epsilon
epsilon_end = 0.0                    # Final epsilon
neps = int(1.*nsteps)               # Number of steps to decay epsilon

learning_rate = 2e-3                 # Initial learning rate
lr_end = 0                           # Final learning rate
nlr = neps                           # Steps to decay learning rate
decay_rate = 0.99                    # Decay factor for RMSProp 

nhidden = 200                        # Number of hidden layers for estimators

init_moves = 2000                    # Upper bound on random number of moves to take initially
nwindow = 2                          # Sensing window = last n images in a state

Below are environment-specific parameters. The function "preprocess" should process an observation returned by the environment into a vector for training. For CartPole we simply append a 1 to implement bias in the first layer. 

For visual environments you would typically crop, downsample to 80x80, set color to a single bit (foreground/background), and flatten to a vector. That transformation is already implemented in the Policy Gradient code.

*nfeats* is the dimension of the vector output by *preprocess*.

In [21]:
game_type="CartPole-v0"                 # Model type and action definitions
VALID_ACTIONS = [0, 1]
nactions = len(VALID_ACTIONS)
nfeats = 5                              # There are four state features plus the constant we add

def preprocess(I):                      # preprocess each observation
    """Just append a 1 to the end"""
    return np.append(I.astype(float),1) # Add a constant feature for bias

Here is the Q-estimator class. We use two instances of this class, one for the target estimator, and one for the Q-estimator. The Q function is normally represented as a scalar $Q(x,a)$ where $x$ is the state and $a$ is an action. For ease of implementation, we actually estimate a vector-valued function $Q(x,.)$ which returns the estimated reward for every action. The model here has just a single hidden layer:

<pre>
Input Layer (nfeats) => FC Layer => RELU => FC Layer => Output (naction values)
</pre>

## 1. Implement Q-estimator gradient
Your first task is to implement the
<pre>Estimator.gradient(s, a, y)</pre>
method for this class. **gradient** should compute the gradients wrt weight arrays W1 and W2 into
<pre>self.grad['W1']
self.grad['W2']</pre>
respectively. Both <code>a</code> and <code>y</code> are vectors. Be sure to update only the output layer weights corresponding to the given action vector. 

In [22]:
class Estimator():

    def __init__(self, ninputs, nhidden, nactions):
        """ Create model matrices, and gradient and squared gradient buffers"""
        model = {}
        model['W1'] = np.random.randn(nhidden, ninputs) / np.sqrt(ninputs)   # "Xavier" initialization
        model['W2'] = np.random.randn(nactions, nhidden) / np.sqrt(nhidden)
        self.model = model
        self.grad = { k : np.zeros_like(v) for k,v in model.iteritems() }
        self.gradsq = { k : np.zeros_like(v) for k,v in model.iteritems() }   
        

    def forward(self, s):
        """ Run the model forward given a state as input.
    returns action predictions and the hidden state"""
        h = np.dot(self.model['W1'], s)
        h[h<0] = 0 # ReLU nonlinearity
        rew = np.dot(self.model['W2'], h)
        return rew, h
    
    
    def predict(self, s):
        """ Predict the action rewards from a given input state"""
        rew, h = self.forward(s)
        return rew
    
              
    def gradient(self, s, a, y):
        """ Given a state s, action a and target y, compute the model gradients"""
    ##################################################################################
    ##                                                                              ##
    ## TODO: Compute gradients and return a scalar loss on a minibatch of size npar ##
    ##    s is the input state matrix (ninputs x npar).                             ##
    ##    a is an action vector (npar,).                                            ##
    ##    y is a vector of target values (npar,) corresponding to those actions.    ##
    ##    return: the loss per sample (npar,).                                      ##                          
    ##                                                                              ##
    ## Notes:                                                                       ##
    ##    * If the action is ai in [0,...,nactions-1], backprop only through the    ##
    ##      ai'th output.                                                           ##
    ##    * loss should be L2, and we recommend you normalize it to a per-input     ##
    ##      value, i.e. return L2(target,predition)/sqrt(npar).                     ##
    ##    * save the gradients in self.grad['W1'] and self.grad['W2'].              ##
    ##    * update self.grad['W1'] and self.grad['W2'] by adding the gradients, so  ##
    ##      that multiple gradient steps can be used beteween updates.              ##
    ##                                                                              ##
    ##################################################################################
        loss = 0.0
        rew, h = self.forward(s)
        Q = rew[a, np.arange(a.size)]
        loss = np.sum((Q-y)**2)/np.sqrt(npar)/np.sqrt(npar)
        dQ = (Q-y).reshape((1,-1))*2./np.sqrt(npar)/np.sqrt(npar)  #!!!sign
        mask = np.where(h>0, np.ones_like(h), np.zeros_like(h))
#         self.grad['W2'][a] += (h*dQ).T
        for i, A in enumerate(a):
            self.grad['W2'][A] += (h*dQ).T[i]
        gradh = (dQ*self.model['W2'][a].T*mask)#.reshape((-1,1))
        self.grad['W1'] += np.dot(gradh,s.T)
        return loss
    
    def ex_gradient(self, s, a, y):
        """ for numerical gradient check"""
        grads = { k : np.zeros_like(v) for k,v in self.model.iteritems() }
        loss = 0.0
        rew, h = self.forward(s)
        Q = rew[a, np.arange(a.size)]
        loss = np.sum((Q-y)**2)/np.sqrt(npar)
        dQ = -(y-Q).reshape((1,-1))*2./np.sqrt(npar)  #!!!sign
        mask = np.where(h>0, np.ones_like(h), np.zeros_like(h))
        for i, A in enumerate(a):
            grads['W2'][A] += (h*dQ).T[i]
        gradh = (dQ*self.model['W2'][a].T*mask)#.reshape((-1,1))
        grads['W1'] += np.dot(gradh,s.T)
        return grads, loss
    
    
    def rmsprop(self, learning_rate, decay_rate): 
        """ Perform model updates from the gradients using RMSprop"""
        for k in self.model:
            g = self.grad[k]
            self.gradsq[k] = decay_rate * self.gradsq[k] + (1 - decay_rate) * g*g
            self.model[k] -= learning_rate * g / (np.sqrt(self.gradsq[k]) + 1e-5)
            self.grad[k].fill(0.0)

## 2. Implement $\epsilon$-Greedy Policy

An $\epsilon$-Greedy policy should:
* with probability $\epsilon$ take a uniformly-random action.
* otherwise choose the best action according to the estimator from the given state.

The function below should implement this policy. It should return a matrix A of size (nactions, npar) such that A[i,j] is the probability of taking action i on input j. The probabilities of non-optimal actions should be $\epsilon/{\rm nactions}$ and the probability of the best action should be $1-\epsilon+\epsilon/{\rm nactions}$.

Since the function processes batches of states, the input <code>state</code> is a <code>ninputs x npar</code> matrix, and the returned value should be a <code>nactions x npar</code> matrix. 

In [23]:
def policy(estimator, state, epsilon):
    """ Take an estimator and state and predict the best action.
    For each input state, return a vector of action probabilities according to an epsilon-greedy policy"""
    ##################################################################################
    ##                                                                              ##
    ## TODO: Implement an epsilon-greedy policy                                     ##
    ##       estimator: is the estimator to use (instance of Estimator)             ##
    ##       state is an (ninputs x npar) state matrix                              ##
    ##       epsilon is the scalar policy parameter                                 ##
    ## return: an (nactions x npar) matrix A where A[i,j] is the probability of     ##
    ##       taking action i on input j.                                            ##
    ##                                                                              ##
    ## Use the definition of epsilon-greedy from the cell above.                    ##
    ##                                                                              ##
    ##################################################################################

    A = np.zeros((nactions, npar))
    A += epsilon/nactions
    actions = np.argmax(estimator.predict(state), axis=0) #orig naction by npar
    for i, act in enumerate(actions):
        A[act,i] += 1-epsilon
    
    return A

This routine copies the state of one estimator into another. Its used to update the target estimator from the Q-estimator.

In [24]:
def update_estimator(to_estimator, from_estimator, window, istep):
    """ every <window> steps, Copy model state from from_estimator into to_estimator"""
    if (istep % window == 0):
        for k in from_estimator.model:
            np.copyto(to_estimator.model[k], from_estimator.model[k])

## 3. Implement "Asynchronous Threads"

Don't try that in Python!! Actually all we do here is create an array of environments and advance each one a random number of steps, using random actions at each step. Later on we will make *synchronous* updates to all the environments, but the environments (and their gradient updates) should remain uncorrelated. This serves the same goal as asynchronous updates in paper [2], or experience replay in paper [1].

In [25]:
block_reward = 0.0;
total_epochs = 0;
   
# Create estimators
q_estimator = Estimator(nfeats*nwindow, nhidden, nactions)
target_estimator = Estimator(nfeats*nwindow, nhidden, nactions)

# The epsilon and learning rate decay schedules
epsilons = np.linspace(epsilon_start, epsilon_end, neps)
learning_rates = np.linspace(learning_rate, lr_end, nlr)

# Initialize the games
print("Initializing games..."); sys.stdout.flush()
envs = np.empty(npar, dtype=object)
state = np.zeros([nfeats * nwindow, npar], dtype=float)
new_state = np.zeros([nfeats * nwindow, npar], dtype=float)
rewards = np.zeros([npar], dtype=float)
dones = np.empty(npar, dtype=int)
actions = np.zeros([npar], dtype=int)


for i in range(npar):
    envs[i] = gym.make(game_type)
    n_step = np.random.randint(nwindow, init_moves)
    for j in xrange(n_step):
        action = np.random.choice(VALID_ACTIONS)
        observation, reward, done, info = envs[i].step(action)
        rewards[i] = reward
        state[:,i] = np.concatenate((state[nfeats:,i], preprocess(observation)))
        block_reward += reward
        if done:
            observation = envs[i].reset()
            state[:,i] = np.concatenate((np.zeros_like(state[nfeats:,i]), preprocess(observation)))
            total_epochs += 1
            #rewards[i] = 0
            
    
   
    ##################################################################################
    ##                                                                              ##
    ## TODO: Advance each environment by a random number of steps, where the number ##
    ##       of steps is sampled uniformly from [nwindow, init_moves].              ##
    ##       Use random steps to advance.                                           ## 
    ##                                                                              ##
    ## Update the total reward and total epochs variables as you go.                ##
    ## If an environment returns done=True, reset it and increment the epoch count. ##
    ##                                                                              ##
    ##################################################################################
       

Initializing games...


[2016-11-22 12:59:16,202] Making new env: CartPole-v0
[2016-11-22 12:59:16,238] Making new env: CartPole-v0
[2016-11-22 12:59:16,249] Making new env: CartPole-v0
[2016-11-22 12:59:16,265] Making new env: CartPole-v0
[2016-11-22 12:59:16,305] Making new env: CartPole-v0
[2016-11-22 12:59:16,331] Making new env: CartPole-v0
[2016-11-22 12:59:16,376] Making new env: CartPole-v0
[2016-11-22 12:59:16,421] Making new env: CartPole-v0
[2016-11-22 12:59:16,463] Making new env: CartPole-v0
[2016-11-22 12:59:16,496] Making new env: CartPole-v0
[2016-11-22 12:59:16,523] Making new env: CartPole-v0
[2016-11-22 12:59:16,566] Making new env: CartPole-v0
[2016-11-22 12:59:16,600] Making new env: CartPole-v0
[2016-11-22 12:59:16,647] Making new env: CartPole-v0
[2016-11-22 12:59:16,661] Making new env: CartPole-v0
[2016-11-22 12:59:16,681] Making new env: CartPole-v0


## 4. Implement Deep Q-Learning
In this cell you actually implement the algorithm. We've given you comments to define all the steps. You should also add book-keeping steps to keep track of the loss, reward and number of epochs (where env.step() returns done = true). 

In [26]:
from cs294_129.gradient_check import eval_numerical_gradient
def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))
ss, aa, yy = np.array([state[:,0]]).T, np.array([actions[0]]), np.array([y[0]])
ss, aa, yy = state, actions, y
grads, loss = q_estimator.ex_gradient(ss, aa, yy)
print loss, q_estimator.gradient(ss, aa, yy)
for param_name in grads:
    f = lambda W: q_estimator.ex_gradient(ss, aa, yy)[1]
    param_grad_num = eval_numerical_gradient(f, q_estimator.model[param_name], verbose=False)
#     print param_grad_num
#     print grads[param_name]
    #print '%s max relative mag: %e' % (param_name, rel_error(param_grad_num, np.zeros_like(param_grad_num)))
    print '%s max relative error: %e' % (param_name, rel_error(param_grad_num, grads[param_name]))

3041.57924617 760.394811542
W2 max relative error: 5.861924e-06
W1 max relative error: 1.345519e-03


In [249]:
env=envs[0]
i=0
done = False
_ = env.reset()
while not done and i < 100:
    i+=1
    observation, reward, done, info = env.step(np.random.choice(VALID_ACTIONS))
    print reward, done
print i
(1-0.99**22)/0.01

1.0 False
1.0 False
1.0 False
1.0 False
1.0 False
1.0 False
1.0 False
1.0 False
1.0 False
1.0 False
1.0 True
11


19.83694104609541

In [27]:
t0 = time.time()
block_loss = 0.0
last_epochs=0

for istep in np.arange(nsteps): 
    if (render): envs[0].render()
    epsilon = epsilons[istep] if istep < neps else epsilon_end
    learning_r = learning_rates[istep] if istep < nlr else lr_end
    update_estimator(target_estimator, q_estimator, target_window, istep)

    A = policy(q_estimator, state, epsilon)
    done_ind = []
    for i, env in enumerate(envs):
        actions[i] = np.random.choice(VALID_ACTIONS, p=A[:,i])
        #actions[i] = np.random.choice(VALID_ACTIONS)
        #print A[:,i], actions[i]
        observation, reward, done, info = env.step(actions[i])
        rewards[i] = reward
        block_reward += reward
        new_state[:,i] = np.concatenate((state[nfeats:,i], preprocess(observation)))
        if done:
            observation = envs[i].reset()
            new_state[:,i] = np.concatenate((preprocess(observation), preprocess(observation)))
            #new_state[:,i] = np.concatenate((np.zeros_like(state[nfeats:,i]), preprocess(observation)))
            total_epochs += 1
            #rewards[i] = 0
            done_ind.append(i)
    Q_score = target_estimator.predict(new_state)
    target = np.amax(Q_score, axis=0)
    y = rewards + discount_factor*target
    #print y[done_ind], rewards[done_ind]
    y[done_ind] = rewards[done_ind]
    loss = q_estimator.gradient(state, actions, y)
    block_loss += loss
    q_estimator.rmsprop(learning_r, decay_rate)

    state = new_state


    #########################################################################
    ## TODO: Implement Q-Learning                                          ##
    ##                                                                     ##
    ## At high level, your code should:                                    ##
    ## * Update epsilon and learning rate.                                 ##
    ## * Update target estimator from Q-estimator if needed.               ##
    ## * Get the next action probabilities for the minibatch by running    ##
    ##   the policy on the current state with the Q-estimator.             ##
    ## * Then for each environment:                                        ##
    ##     ** Pick an action according to the action probabilities.        ##
    ##     ** Step in the gym with that action.                            ##
    ##     ** Process the observation and concat it to the last nwindow-1  ##
    ##        processed observations to form a new state.                  ##
    ## Then for all environments (vectorized):                             ##
    ## * Predict Q-scores for the new state using the target estimator.    ##
    ## * Compute new expected rewards using those Q-scores.                ##
    ## * Using those expected rewards as a target, compute gradients and   ##
    ##   update the Q-estimator.                                           ##
    ## * Step to the new state.                                            ##
    ##                                                                     ##
    #########################################################################

    t = time.time() - t0
    if (istep % printsteps == 0):
        #print A
        #print q_estimator.predict(state)
        #print state
        print("step {:0d}, time {:.1f}, loss {:.8f}, epochs {:0d}, reward/epoch {:.5f}".format(
                istep, t, block_loss/printsteps, total_epochs, block_reward/np.maximum(1,total_epochs-last_epochs)))
        last_epochs = total_epochs
        block_reward = 0.0
        block_loss = 0.0

step 0, time 0.0, loss 0.00122135, epochs 881, reward/epoch 22.08740
step 1000, time 1.0, loss 0.06109310, epochs 1594, reward/epoch 22.44039
step 2000, time 2.0, loss 0.05042740, epochs 2371, reward/epoch 20.59202
step 3000, time 3.0, loss 0.04691446, epochs 3165, reward/epoch 20.15113
step 4000, time 3.9, loss 0.05152649, epochs 4016, reward/epoch 18.80141
step 5000, time 4.9, loss 0.05441860, epochs 4910, reward/epoch 17.89709
step 6000, time 5.9, loss 0.05322341, epochs 5877, reward/epoch 16.54602
step 7000, time 6.9, loss 0.04559015, epochs 6890, reward/epoch 15.79467
step 8000, time 7.8, loss 0.04220275, epochs 7994, reward/epoch 14.49275
step 9000, time 8.8, loss 0.03575175, epochs 9239, reward/epoch 12.85141
step 10000, time 9.9, loss 0.05857377, epochs 10711, reward/epoch 10.86957


Let's save the model now. 

In [56]:
pickle.dump(q_estimator.model, open("cartpole_q_estimator.p", "wb"))


You can reload the model later if needed:

In [57]:
test_estimator = Estimator(nfeats*nwindow, nhidden, nactions)
test_estimator.model = pickle.load(open("cartpole_q_estimator.p", "rb"))

And animate the model's performance. 

In [39]:
state0 = state[:,0]
for i in np.arange(200):
    envs[0].render()
    preds = test_estimator.predict(state0)
    iaction = np.argmax(preds)
    obs, _, done0, _ = envs[0].step(VALID_ACTIONS[iaction])
    state0 = np.concatenate((state0[nfeats:], preprocess(obs)))
    if (done0): envs[0].reset()
    

GLXInfoException: pyglet requires an X server with GLX

So there we have it. Simple 1-step Q-Learning can solve easy problems very fast. Note that environments that produce images will be much slower to train on than environments (like CartPole) which return an observation of the state of the system. But this model can still train on those image-based games - like Atari games. It will take hours-days however. It you try training on visual environments, we recommend you run the most expensive step - rmsprop - less often (e.g. every 10 iterations). This gives about a 3x speedup. 

## Optional
Do **one** of the following tasks:
* Adapt the DQN algorithm to another environment - it can use direct state observations.  Call <code>env.get_action_meanings()</code> to find out what actions are allowed. Summarize training performance: your final average reward/epoch, the number of steps required to train, and any modifications to the model or its parameters that you made.
* Try smarter schedules for epsilon and learning rate. Rewards for CartPole increase very sharply (several orders of magnitude) with better policies, especially as epsilon --> 0. Gradients will also change drastically, so the initial learning rate is probably not good later on. Try schedules for decreasing epsilon that allow the model to better adapt. Try other learning rate schedules, or setting learning rate based on average reward. 
* Try a fancier model. e.g. add another hidden layer, or try sigmoid non-linearities.